# Root Ad Auction Dataset
## Team: The Puffy Shirts

the data set:

    30 days of CSV files    
    rows are winning bids in a second-price auction
    contains information on the target user/platform, the UTC timestamp, the app, the exchange, etc...
    

# Challenges of this data set

Large data set:

- 1.42 GB as .zip / 8.19 GB as .csv
    
- 36.3M rows, 26 columns
    
Highly unbalanced categorical classification problem:

- 1.57M clicks (4.3% of rows)
    
- 2,109 installs (0.0058% of rows, 0.13% of clicks)

# Preprocessing

## First look at the data

In [ ]:
import os, sys, time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.rcParams['figure.dpi'] = 240 # fix high-dpi display scaling issues

sys.path.append(os.getcwd()) # add cwd to path

from zip_codes import ZC # zip code database
import load_file as lf # file i/o
import myplots as mp # my plotting functions
zc = ZC('') # initialize zip code class

In [ ]:
fname_27 = '2019-04-27.csv'
data_dir = r'C:\PythonBC\RootData'

In [ ]:
start = time.time()
df_27 = pd.read_csv( os.path.join(data_dir, fname_27))
end = time.time()
print(f'loading {fname_27} with default pd.read_csv() settings takes'
      + f' {end-start:3.2f} seconds and {lf.mem_usage(df_27)} of RAM')

some columns can be immediately dropped:
- auction_id (meaningless string)
- platform_os (all 'Android' or '-1')
- app_bundle (all the same)
- year (UTC time)
- month (UTC time)
- day (UTC time)
- day_of_week (UTC time, incorrectly calculated)
- hour (UTC time)
- creative_size (redundant with creative_type)

most columns are categorical with only a few unique values:
- inventory_source (only 4 possible values)
- category
- platform_bandwidth
- platform_carrier
- platform_device_make (only 5 possible values)
- platform_device_model
- platform_device_screen_size
- geo_zip

some are boolean:
- inventory_interstitial
- rewarded
- clicks
- installs

some columns need additional work:
- geo_zip ('43212.0' >>> '43212')
- category & segments are unsorted string lists

some columns have multiple nan values:
- platform_carrier
- platform_device_make
- platform_device_model
- platform_device_screen_size

The function lf.load_data() drops the useless columns and cleans & downcasts the others:

In [ ]:
start = time.time()
#df_27 = lf.load_wrapper(fname=fname_27, data_dir=data_dir)
df_27 = lf.load_data(fname=fname_27, data_dir=data_dir)
end = time.time()
print(f'loading {fname_27} with downcasting takes {end-start:3.2f}'
      + f' seconds and {lf.mem_usage(df_27)} of RAM')

Using these tricks we reduce the RAM usage by a factor of **9.1**, but increase the load time by 43%. With this RAM reduction we can comfortably load the entire dataset into memory (~ 850 MB).

We can quickly save the processed dataframe to disk using pd.to_parquet():

In [ ]:
start_save = time.time()
lf.temp_save(df_27, fname=os.path.join(data_dir, fname_27.split('.')[0]+'.gzip'))
end_save = time.time()
df_27 = lf.temp_load( os.path.join(data_dir, fname_27.split('.')[0]+'.gzip') )
end_load = time.time()

print(f'saving to .gzip takes {end_save-start_save:3.2f}'
      + f' seconds and loading from .gzip takes {end_load-end_save:3.2f} seconds')

Because parquet is so fast, it makes sense to pre-process all the data and save it to disk for later.

## Using the geo_zip column

We can use a zip code database (https://www.unitedstateszipcodes.org/zip-code-database/) to get local information for each bid:

In [ ]:
myzip = ['43210']
print(f'zip code {myzip[0]}:')
print( f'   state: {zc.zip_to_state_2(myzip)[0]}' )
print( f'   county: {zc.zip_to_county_2(myzip)[0]}' )
print( f'   timezone: {zc.zip_to_tz_2(myzip)[0]}' )
print( f'   coordinates: {zc.zip_to_lat_2(myzip)[0], zc.zip_to_lon_2(myzip)[0]}')

Now that we have the local time zone, we can shift bid_timestamp_utc to local time:

In [ ]:
print(f'UTC timestamp = {df_27.bid_timestamp_utc.iloc[0]}')
print(f'local timestamp = {df_27.bid_timestamp_utc.iloc[0].tz_convert("America/New_York")}')

This process is slow, so run it on each .csv file separately and combine the results at the end.

With the local timestamp, we can extract quantities like hour, day and day_of_week.

The functions reshape_files() and local_hour_creator() do the aforementioned preprocessing and save each column as a .gzip file. This step takes about 30-40 minutes on a laptop.

# Visualization: maps

Now that we have all the data in memory, we can make visualizations

## Choropleths

In [ ]:
import folium
state_geo = 'us-states.json' # local copy of json file

# load relevant data from disk
data_dir = r'C:\PythonBC\RootData'
df_clicks = lf.temp_load( os.path.join(data_dir, 'clicks.gzip')  )
df_state = lf.temp_load( os.path.join(data_dir, 'state.gzip')  )
df_installs = lf.temp_load( os.path.join(data_dir, 'installs.gzip')  )
frames = [df_state, df_clicks, df_installs]
df = pd.concat(frames, axis=1)

# number of clicks per state
dfstates = df.groupby('state').sum()['clicks'].to_frame()
dfstates.reset_index(level=0, inplace=True)

# number of bids per state
dfstates2 = df.groupby('state').count()['clicks'].to_frame()
dfstates2.reset_index(level=0, inplace=True)
dfstates2.rename(index=str, columns={"state": "state", "clicks": "bids"})

# number of installs per state
dfstates3 = df.groupby('state').sum()['installs'].to_frame()
dfstates3.reset_index(level=0, inplace=True)
dfstates3.rename(index=str, columns={"state": "state", "clicks": "installs"})

# build new dataframe
bids = dfstates2.clicks.values
clicks = dfstates.clicks.values
installs = np.asarray(dfstates3.installs)
state = dfstates.state.values
clickrate = 100*np.divide(clicks, bids)
installrate = 100*np.divide(installs, bids)
frames = {"state": state, "bids": bids, "clicks": clicks, "installs": installs, "clickrate":clickrate, "installrate": installrate}
df_rate = pd.DataFrame(data=frames)
df_rate_nonzero = df_rate[df_rate.clickrate > 0]

In [ ]:
clickrate_m = folium.Map(location=[39.50, -98.35], zoom_start=4) # lower 48
folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=df_rate_nonzero,
    columns=['state', 'clickrate'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='click rate (%)'
).add_to(clickrate_m)
folium.LayerControl().add_to(clickrate_m)
clickrate_m

In [ ]:
installrate_m = folium.Map(location=[39.50, -98.35], zoom_start=4) # lower 48
folium.Choropleth(
    geo_data=state_geo,
    name='choropleth',
    data=df_rate_nonzero,
    columns=['state', 'installrate'],
    key_on='feature.id',
    fill_color='YlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='install rate (%)'
).add_to(installrate_m)
folium.LayerControl().add_to(installrate_m)
installrate_m

## Heatmaps

In [ ]:
from folium.plugins import HeatMap

zipc = lf.temp_load( os.path.join(data_dir, 'geo_zip.gzip'))
click = lf.temp_load( os.path.join(data_dir, 'clicks.gzip'))
df = pd.concat([zipc,click],axis=1)

dftest = df.query('clicks == True')
dftest = pd.DataFrame(dftest.geo_zip.value_counts())
df = pd.DataFrame(df.geo_zip.value_counts())
df['latitude'] = zc.zip_to_lat_2(df.index)
df['longitude'] = zc.zip_to_lon_2(df.index)

dfratio = dftest/df
dfratio['latitude'] = zc.zip_to_lat_2(dfratio.index)
dfratio['longitude'] = zc.zip_to_lon_2(dfratio.index)

df=df.dropna()
dfratio=dfratio.dropna()

df = df[['latitude','longitude','geo_zip']]
df_copy = df.copy()

dfratio = dfratio[['latitude','longitude','geo_zip']]
dfratio_copy = dfratio.copy()

###  Heatmap of ads

In [ ]:
m = folium.Map(location=[38.5, -100],zoom_start=4)
HeatMap(data=df_copy[['latitude', 'longitude', 'geo_zip']].groupby(['latitude', 'longitude']).\
        sum().reset_index().values.tolist(), radius=6, max_zoom=13).add_to(m)
m

### Heatmap of (clicks/bids)

In [ ]:
mratio = folium.Map(location=[38.5, -100],zoom_start=4)
HeatMap(data=dfratio_copy[['latitude', 'longitude', 'geo_zip']].groupby(['latitude', 'longitude']).\
        sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(mratio)
mratio

# Building a classifier

In [ ]:
from IPython.display import Image

categorical encoding

how you balance the data 

metrics for evaluating model fit





trained on 1st 3 weeks:
under sampled:
![title](img\RandomUnderSampler_firstweek.png)

over sampled:
![title](img\RandomOverSampler_firstweek.png)

same model applied to the last week:

under sampled:
![title](img\RandomUnderSampler_lastweek.png)

over sampled:
![title](img\RandomOverSampler_lastweek.png)

